## Identifcation of IS_HCP and  Taxonomy 

### Import required libraries ###

In [1]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import xgboost as xb
import gc

### Create a class which will be used for feature engineering and preprocessing ### 
* fit_transfrom method is used to apply feature engineering for train data
* transform method is used to apply feature engineering for test data
* This class makes sure that same columns are present after preprocessing for test and train data

### Following colummns are dropped as they ae not relavent/or part of prediction: ###

* USERCITY,BIDREQUESTIP --> ZIPCODE Provides similar information, hence this information is kind if duplicate and doesn't look related to HCP
* TAXONOMY --> This is to be predicted 
* USERAGENT --> Browser infor not related to HCP
* CHANNELTYPE -->  Is Website, there are no other channel types hence removed
* ID --> Doesn't have correlation with HCP
* USERPLATFORMUID --> not related to HCP


### Following colummns are one hot encoded: ###
* URL only the website name is extracted from the whole website and is one hot encoded
* PLATFORMTYPE is one hot encoded
* DEVICETYPE is one hot encoded


### Following column is label encoded ###

* KEYWORDS--> All the keywords are extracted using split(), then columns are created with Keyword name and if the Keyword exist for the row the 1 is added for the column as 0 is added for the column



In [2]:
class feature_enineering:
    
    def __init__(self):
        self.columns = list()
        self.all_columns = list()
        self.taxonomy = list()
        self.taxo_dict = dict()

        
    def encode_uid(self,x):
        
        if(x in self.UID_dict.keys()):
            return self.UID_dict[x]
        else:
            return 0
        
    def encode_taxo(self,x):
        
        if(x in self.taxo_dict.keys()):
            return self.taxo_dict[x]
        else:
            return 5

        
    def split_data(self,x):
        
        data  = x.split('/')
        if len(data)>=3:
            return data[2]
        else:
            return data[0]
            
        
    
    def fit_transform(self,df):
    
        
        # split medical keywords, and create a diction of number of words and keywords 
        
        key_data= dict()
        for i,keys in enumerate(df["KEYWORDS"]):
            for key in  keys.split("|"):
                if key in key_data.keys():
                    key_data[key] +=1
                else:
                    key_data[key] =0
                    
        key_data.pop('')
        
        key_final = dict()
        for key in key_data.keys():
            if key_data[key] < 100 or key_data[key]  > 55000:
                continue
            else:
                key_final[key]= key_data[key]
                
        self.columns = key_final.keys()
        

        indexs = df.index
        
        df_dict = dict()
        i= 0
        all_key =  df["KEYWORDS"]
        for key in  list(self.columns):
            df_dict[key]= list()
            i +=1
            #print(i)
            for index in indexs:
                if(key in all_key[index]):
                    #df.loc[index,key] =1
                    df_dict[key].append(1)

                else:
                    #df.loc[index,key] =0
                    df_dict[key].append(0)
                    
        new_df = pd.DataFrame(df_dict, index = indexs)
        df = pd.concat([df,new_df], axis=1)
            
        del new_df, df_dict
        gc.collect()
            
            
        ids = df["USERPLATFORMUID"].unique()
        self.UID_dict = dict()
        for i, uid in enumerate(ids,start=1):
            self.UID_dict[uid]=i
            
        df["USERPLATFORMUID"] = df["USERPLATFORMUID"].apply(self.encode_uid)
        df["URL"]= df["URL"].apply( self.split_data)
        
        
        taxonomy = df["TAXONOMY"].values
        
        self.taxo_dict = dict()
        i = 0
        for  tax in taxonomy:

            if(tax not in self.taxo_dict.keys() ):
                self.taxo_dict[tax]=i
                i+=1

        df["TAX_dig"]= df["TAXONOMY"].apply(self.encode_taxo)
    
        
        df = df.drop("USERCITY", axis=1)
        df = df.drop("BIDREQUESTIP", axis=1)
        df = df.drop("TAXONOMY", axis=1)
        df = df.drop("USERAGENT", axis=1)
        df = df.drop("CHANNELTYPE", axis=1)
        df = df.drop("KEYWORDS", axis=1)
        df = df.drop("ID", axis=1)
        df = df.drop("USERPLATFORMUID", axis=1)

            
            
        
        df["USERZIPCODE"].fillna('000000', inplace=True)
        df["USERZIPCODE"] = df["USERZIPCODE"].astype(int)
        df = pd.get_dummies(df, columns=["URL", "PLATFORMTYPE", "DEVICETYPE"], dtype=int,  sparse=True)
        
        self.all_columns = df.columns
        
        return df,self.taxo_dict

            


    def transform(self, df):
        
        indexs = df.index
        
        df_dict = dict()
        i= 0
        all_key =  df["KEYWORDS"]
        for key in  list(self.columns):
            df_dict[key]= list()
            i +=1
            #print(i)
            for index in indexs:
                if(key in all_key[index]):
                    #df.loc[index,key] =1
                    df_dict[key].append(1)

                else:
                    #df.loc[index,key] =0
                    df_dict[key].append(0)
                    
        new_df = pd.DataFrame(df_dict, index = indexs)
        df = pd.concat([df,new_df], axis=1)
        
        del new_df, df_dict
        gc.collect()
        
        df["USERPLATFORMUID"] = df["USERPLATFORMUID"].apply(self.encode_uid)
        df["URL"]= df["URL"].apply(self.split_data)
        
        
        df["TAX_dig"]= df["TAXONOMY"].apply(self.encode_taxo)
  
        
        df = df.drop("USERCITY", axis=1)
        df = df.drop("BIDREQUESTIP", axis=1)
        df = df.drop("TAXONOMY", axis=1)
        df = df.drop("USERAGENT", axis=1)
        df = df.drop("CHANNELTYPE", axis=1)
        df = df.drop("KEYWORDS", axis=1)
        df = df.drop("ID", axis=1)
        df = df.drop("USERPLATFORMUID", axis=1)

        
        df["USERZIPCODE"].fillna('000000', inplace=True)
        df["USERZIPCODE"] = df["USERZIPCODE"].astype(int)
        df = pd.get_dummies(df, columns=["URL", "PLATFORMTYPE", "DEVICETYPE"], dtype=int,  sparse=True)
        
        transfomed_col = set(df.columns)
        col_to_add = set(self.all_columns) - transfomed_col
        col_to_remove = transfomed_col - set(self.all_columns)
        
        for col in col_to_add:
            df[col] = 0
            
        for col in col_to_remove:
            df.drop(col, axis =1, inplace = True )
            
        
      
        df =df.reindex(columns=list(self.all_columns))
            
        
        return df

### Download the train data set and check basic info ##

In [3]:
dtype_dict = {'USERZIPCODE': object}
df = pd.read_csv("Doceree-HCP_Train.csv",encoding = "ISO-8859-1", dtype=dtype_dict)
df = df[pd.notna(df["IS_HCP"])]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 113936 entries, 0 to 113936
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ID               113936 non-null  int64  
 1   DEVICETYPE       113936 non-null  object 
 2   PLATFORM_ID      113936 non-null  int64  
 3   BIDREQUESTIP     113936 non-null  object 
 4   USERPLATFORMUID  113932 non-null  object 
 5   USERCITY         107577 non-null  object 
 6   USERZIPCODE      109344 non-null  object 
 7   USERAGENT        113934 non-null  object 
 8   PLATFORMTYPE     113936 non-null  object 
 9   CHANNELTYPE      113936 non-null  object 
 10  URL              113936 non-null  object 
 11  KEYWORDS         113936 non-null  object 
 12  TAXONOMY         32313 non-null   object 
 13  IS_HCP           113936 non-null  float64
dtypes: float64(1), int64(2), object(11)
memory usage: 13.0+ MB


In [43]:
df.head()

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS,TAXONOMY,IS_HCP
0,1001,Desktop,2,170.173.0.22,6974dcaa-f932-480e-9fb5-c52e20e1393a,Portland,97206,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0
1,1002,Desktop,2,65.216.253.25,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,Arlington,22202,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/deci...,Bone Marrow|Radiography|Chronic|Oncology|Psych...,NaN,0.0
2,1003,Desktop,2,66.232.79.22,a698de4b-e200-46dd-b5fb-40402175ae18,New Meadows,83654,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0
3,1004,Desktop,3,137.54.125.246,45967533-75c8-4fbd-a00c-e6ff20447aaa,NaN,229114624,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://globalrph.com/medcalcs/warfarin-mainte...,Dental|Total|Clinical|Pharmacology|Physicians|...,2084P0800X,1.0
4,1005,Mobile,7,174.202.231.99,a17e25be-532d-4cf5-b916-9308c8c3961f,Houston,77008,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,Online Medical Journal,Website,https://www.cureus.com/articles/58184-a-review...,Critical Care|Emergency Medicine|General Pract...,NaN,0.0


### Split data into train and test and use try to predict  IS_HCP inside train data ###

In [4]:
df_input = df.drop("IS_HCP", axis=1)
df_result = df["IS_HCP"]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df_input, df_result, test_size=0.33)

In [6]:
fe= feature_enineering()

In [7]:
X_train,taxonomy_dict = fe.fit_transform(X_train)

In [8]:
X_train.head()

,PLATFORM_ID,USERZIPCODE,Shock,Myocardial Ischemia,Total,Psychiatry,Arthritis,General,Cardiovascular,Induced,...,URL_www.thecardiologyadvisor.com,PLATFORMTYPE_Electronic Health Record,PLATFORMTYPE_Medical News Platform,PLATFORMTYPE_Online Learning Portal,PLATFORMTYPE_Online Medical Journal,PLATFORMTYPE_Tele-medicine Platform,DEVICETYPE_Desktop,DEVICETYPE_Mobile,DEVICETYPE_Tablet,DEVICETYPE_Unknown
68109,2,55901,1,1,1,1,1,1,1,1,...,1,0,0,0,1,0,1,0,0,0
107103,2,83401,0,0,0,1,1,1,1,0,...,0,0,0,0,1,0,1,0,0,0
657,2,50266,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,1,0,0,0
53977,2,42001,0,0,1,1,0,1,0,0,...,1,0,0,0,1,0,0,0,1,0
77587,2,70471,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [9]:
X_test = fe.transform(X_test)

In [10]:
set(X_test.columns) - set(X_train.columns)

set()

In [11]:
X_test.head()

,PLATFORM_ID,USERZIPCODE,Shock,Myocardial Ischemia,Total,Psychiatry,Arthritis,General,Cardiovascular,Induced,...,URL_www.thecardiologyadvisor.com,PLATFORMTYPE_Electronic Health Record,PLATFORMTYPE_Medical News Platform,PLATFORMTYPE_Online Learning Portal,PLATFORMTYPE_Online Medical Journal,PLATFORMTYPE_Tele-medicine Platform,DEVICETYPE_Desktop,DEVICETYPE_Mobile,DEVICETYPE_Tablet,DEVICETYPE_Unknown
34483,2,43016,0,0,0,1,1,1,1,0,...,0,0,0,0,1,0,1,0,0,0
31468,7,90620,0,0,1,1,0,1,0,0,...,0,0,0,0,1,0,1,0,0,0
42257,2,37190,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
39733,2,77018,0,0,0,1,1,1,1,0,...,0,0,0,0,1,0,1,0,0,0
45846,2,11368,0,0,0,1,0,1,1,0,...,0,0,0,0,1,0,1,0,0,0


### Train and test split in ratio 67:33  (using train data)###
 ### Test result is predicted  and  a classification report is created for it ###

In [12]:
X_train_case1 = X_train.drop('TAX_dig', axis = 1)
X_test_case1 = X_test.drop('TAX_dig', axis = 1)

model = xb.XGBClassifier()
model.fit(X_train_case1,y_train)
y_predict =model.predict(X_test_case1)
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98     26314
         1.0       0.97      0.94      0.95     11285

    accuracy                           0.97     37599
   macro avg       0.97      0.96      0.97     37599
weighted avg       0.97      0.97      0.97     37599



### 97% accuaracy has been achieved now lets predict on thetest Set ###
* Train and Test data is downloded in separate DF
* The DFs are feature engineered
* Model is trained using complete train data and  Test data is used to predict IS_HCP

In [14]:
dtype_dict = {'USERZIPCODE': object}
df_train = pd.read_csv("Doceree-HCP_Train.csv",encoding = "ISO-8859-1", dtype=dtype_dict)
df_train = df_train[pd.notna(df_train["IS_HCP"])]
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 113936 entries, 0 to 113936
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ID               113936 non-null  int64  
 1   DEVICETYPE       113936 non-null  object 
 2   PLATFORM_ID      113936 non-null  int64  
 3   BIDREQUESTIP     113936 non-null  object 
 4   USERPLATFORMUID  113932 non-null  object 
 5   USERCITY         107577 non-null  object 
 6   USERZIPCODE      109344 non-null  object 
 7   USERAGENT        113934 non-null  object 
 8   PLATFORMTYPE     113936 non-null  object 
 9   CHANNELTYPE      113936 non-null  object 
 10  URL              113936 non-null  object 
 11  KEYWORDS         113936 non-null  object 
 12  TAXONOMY         32313 non-null   object 
 13  IS_HCP           113936 non-null  float64
dtypes: float64(1), int64(2), object(11)
memory usage: 13.0+ MB


In [15]:
dtype_dict = {'USERZIPCODE': object}
df_test = pd.read_csv("Doceree-HCP_Test.csv",encoding = "ISO-8859-1", dtype=dtype_dict)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28493 entries, 0 to 28492
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               28493 non-null  int64 
 1   DEVICETYPE       28493 non-null  object
 2   PLATFORM_ID      28493 non-null  int64 
 3   BIDREQUESTIP     28493 non-null  object
 4   USERPLATFORMUID  28493 non-null  object
 5   USERCITY         26934 non-null  object
 6   USERZIPCODE      27378 non-null  object
 7   USERAGENT        28492 non-null  object
 8   PLATFORMTYPE     28493 non-null  object
 9   CHANNELTYPE      28493 non-null  object
 10  URL              28493 non-null  object
 11  KEYWORDS         28493 non-null  object
dtypes: int64(2), object(10)
memory usage: 2.6+ MB


In [16]:
fe_engg = feature_enineering()
df_train_y = df_train["IS_HCP"]
df_train_x = df_train.drop("IS_HCP", axis =1)

In [17]:
df_train_x, _ = fe_engg.fit_transform(df_train_x)

In [18]:
df_test["TAXONOMY"] =0
df_test_x= fe_engg.transform(df_test)

In [19]:
df_train_x.drop('TAX_dig', inplace = True, axis = 1)
df_test_x.drop('TAX_dig', inplace = True, axis = 1)

In [20]:
model = xb.XGBClassifier()
model.fit(df_train_x,df_train_y)
y_predict =model.predict(df_test_x)

In [21]:
len(y_predict)

28493

### Create Submission file and Upload ###

In [22]:
df_submission= pd.read_csv("Doceree-HCP_Sample_Submission.csv",encoding = "ISO-8859-1")
df_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28493 entries, 0 to 28492
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   ID      28493 non-null  int64
 1   IS_HCP  28493 non-null  int64
dtypes: int64(2)
memory usage: 445.3 KB


In [23]:
df_submission['IS_HCP'] = y_predict

In [24]:
df_submission.to_csv("Is_HCP_output.csv", index=False)

### Check taxonomy ###
* TAXONOMY is first label encoded , key value pair is stored
* Train Data Frame which has  IS_HCP== 1 is trained on a xgboost model to predict numeric labeled TAXONOMY 
* Test Data where IS_HCP is identified as 1 is used to predict labeled TAXONOMY 
* numberic labels of TAXONOMY is converted to text labels 


In [25]:
df_train_tax = df_train[~df_train['TAXONOMY'].isna()]

In [26]:
tax_fe = feature_enineering()
df_train_tax, tax_dict= tax_fe.fit_transform(df_train_tax)


In [27]:
df_train_tax_y = df_train_tax['TAX_dig']
df_train_tax_x = df_train_tax.drop('TAX_dig', axis = 1)

In [28]:
model = xb.XGBClassifier()
model.fit(df_train_tax_x,df_train_tax_y)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [29]:
df_test

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS,TAXONOMY
0,115501,Desktop,2,75.189.231.103,0d5041ff-f0b6-4d1a-9ad7-0a29f7d485b4,Fayetteville,28305,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6...,Online Medical Journal,Website,https://www.clinicaladvisor.com/home/features/...,Family Practice|Drainage|Clinical|Dermatology|...,0
1,115502,Mobile,2,24.101.33.158,c8396dd0-969f-4d99-a40b-b7bb1f516154,Conneaut Lake,16316,Mozilla/5.0 (iPhone; CPU iPhone OS 15_6_1 like...,Online Medical Journal,Website,https://www.ophthalmologyadvisor.com/topics/ca...,General|Clinical|Operative|Medicine|Cardiology...,0
2,115503,Desktop,2,172.118.216.142,3c97a081-6518-43f8-9f26-369759cfb471,Covina,91724,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.psychiatryadvisor.com/author/tori-...,Abortion|Anxiety Disorders|Apnea|False|Trauma|...,0
3,115504,Desktop,7,71.105.120.171,3e2578c8-f794-41af-a38c-c5cfb3c0f014,Brooklyn,11226,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cureus.com/articles/105482-diverti...,Health|Male|Neurological Surgery|Otolaryngolog...,0
4,115505,Desktop,2,73.82.211.73,ec2ae7ce-6a8c-4156-98a7-07203e60f483,Marietta,30062,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.renalandurologynews.com/home/confe...,chronic kidney disease|pain|nephrology|disease...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28488,143989,Desktop,2,69.202.233.241,78ce4bbe-3885-4c14-b945-a0ea1e4574f4,Brooklyn,11215,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,Online Medical Journal,Website,https://www.renalandurologynews.com/home/news/...,Transplantation|Psychiatry|Heart Failure|Angio...,0
28489,143990,Desktop,7,75.4.190.65,a53799e1-b279-40cf-b467-11b4d659e325,Miami,33178,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.cureus.com/publish/articles/144818...,small|hospitals|nephrology|emergency medicine|...,0
28490,143991,Desktop,7,137.52.180.45,acc732a8-2ee9-4e2c-9933-70e34db48101,Fort Lauderdale,33314,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cureus.com/registrations/continue?...,small|hospitals|nephrology|emergency medicine|...,0
28491,143992,Desktop,8,66.249.66.4,dce21294-b105-4abb-b145-4e62d71def44,NaN,NaN,Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Bu...,Online Medical Journal,Website,https://radrounds.com/radiology-case-images-te...,the|disease|pain|radiology|lung abscess|dyspar...,0


In [30]:
df_submission

,ID,IS_HCP
0,115501,0
1,115502,1
2,115503,0
3,115504,0
4,115505,1
...,...,...
28488,143989,0
28489,143990,0
28490,143991,0
28491,143992,0


In [31]:
df_test_all = pd.concat([df_test,df_submission["IS_HCP"]],axis = 1)

In [32]:
selected_test = df_test_all[df_test_all["IS_HCP"]==1]
selected_test_ids = selected_test['ID']
test_df_hpc_1  = selected_test.drop('IS_HCP', axis=1)

In [33]:

df_test_x= tax_fe.transform(test_df_hpc_1)
df_test_x.drop('TAX_dig', axis = 1)

,PLATFORM_ID,USERZIPCODE,IS_HCP,Dental,Total,Clinical,Pharmacology,Physicians,Health,Drug Therapy,...,URL_www.renalandurologynews.com,URL_www.rheumatologyadvisor.com,URL_www.thecardiologyadvisor.com,PLATFORMTYPE_Electronic Health Record,PLATFORMTYPE_Medical News Platform,PLATFORMTYPE_Online Learning Portal,PLATFORMTYPE_Online Medical Journal,DEVICETYPE_Desktop,DEVICETYPE_Mobile,DEVICETYPE_Tablet
1,2,16316,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
4,2,30062,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
10,10,174024661,0,1,0,1,0,1,1,0,...,0,0,0,0,0,0,1,1,0,0
12,9,94558,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
29,2,46304,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28472,2,91343,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0
28473,2,20003,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
28474,2,49230,0,0,1,1,0,0,1,0,...,1,0,0,0,0,0,1,1,0,0
28476,2,19044,0,0,1,1,0,1,1,0,...,0,0,0,0,0,0,1,1,0,0


In [34]:
df_test_x.shape, df_train_tax_x.shape

((8249, 436), (32313, 435))

In [35]:
df_test_x.drop("TAX_dig", axis = 1, inplace = True)

In [36]:
y_predict =model.predict(df_test_x)

In [37]:
y_predict

array([2, 2, 6, ..., 2, 6, 2], dtype=int64)

In [38]:
IDs = test_df_hpc_1['ID'].values

In [39]:
inv_map = {v: k for k, v in tax_dict.items()}

In [40]:
def set_taxonomy(id):
    if(id in IDs):
        ind = list(IDs).index(id)
        #print(y_predict[ind])
        
        return inv_map[y_predict[ind]]
    
    else:
        return ""
        
        

In [41]:
df_submission["TAXONOMY"] = df_submission["ID"].apply(set_taxonomy)

### Submission file is saved with Taxonomy and IS_HCP ###

In [42]:
df_submission.to_csv("Is_HCP_with_Taxonomy.csv", index=False)